## Analise de texto IMDB utilizando o TensorFlow para atividade de aprofundamento da trilha 04 de DeepLearning.

Anderson rosa cascalho
Link para acesso: https://colab.research.google.com/drive/16ua_Tfw_vyi_wZ7jaqJErrHqbqJxJNCg?usp=sharing

In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np


In [ ]:
imdb = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17473536/17464789 [==============================] - 0s 0us/step


## Tratamento dos dados

In [ ]:
print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))

Training entries: 25000, labels: 25000


In [ ]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [ ]:
len(train_data[0]), len(train_data[1])

(218, 189)

In [ ]:
word_index = imdb.get_word_index()
# No base já foi tratada para e foi transformado os textos em numeros. Aqui é caso quisessemos transformar os numeros inteiros em palavras novamente. 
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

1654784/1641221 [==============================] - 0s 0us/step


In [ ]:
#Aqui estamos deixando os dados com o mesmo tamanho, para não dar diferença no tamanho na entrada aos tensores.
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

In [ ]:
len(train_data[0]), len(train_data[1])

(256, 256)

In [ ]:
train_data[0]

array([   1,   14,   22,   16,   43,  530,  973, 1622, 1385,   65,  458,
       4468,   66, 3941,    4,  173,   36,  256,    5,   25,  100,   43,
        838,  112,   50,  670,    2,    9,   35,  480,  284,    5,  150,
          4,  172,  112,  167,    2,  336,  385,   39,    4,  172, 4536,
       1111,   17,  546,   38,   13,  447,    4,  192,   50,   16,    6,
        147, 2025,   19,   14,   22,    4, 1920, 4613,  469,    4,   22,
         71,   87,   12,   16,   43,  530,   38,   76,   15,   13, 1247,
          4,   22,   17,  515,   17,   12,   16,  626,   18,    2,    5,
         62,  386,   12,    8,  316,    8,  106,    5,    4, 2223, 5244,
         16,  480,   66, 3785,   33,    4,  130,   12,   16,   38,  619,
          5,   25,  124,   51,   36,  135,   48,   25, 1415,   33,    6,
         22,   12,  215,   28,   77,   52,    5,   14,  407,   16,   82,
          2,    8,    4,  107,  117, 5952,   15,  256,    4,    2,    7,
       3766,    5,  723,   36,   71,   43,  530,  4

## Modelo

In [ ]:
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer=tf.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Treinando o modelo

In [ ]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

Configurando o modelo para que rode em 40 épocas e utilize mini lotes de 512 amostras. Estas 40 iterações sobre todas as amostras nos tensores partial_x_train e partial_y_train. Durante o treino, monitore o erro e a acurácia do modelo sobre as 10.000 amostras no conjunto de validação x_val e y_val.

In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Epoch 1/40
30/30 [==============================] - 1s 20ms/step - loss: 0.6922 - accuracy: 0.5840 - val_loss: 0.6904 - val_accuracy: 0.7052
Epoch 2/40
30/30 [==============================] - 1s 19ms/step - loss: 0.6874 - accuracy: 0.6870 - val_loss: 0.6840 - val_accuracy: 0.7450
Epoch 3/40
30/30 [==============================] - 1s 20ms/step - loss: 0.6776 - accuracy: 0.7587 - val_loss: 0.6718 - val_accuracy: 0.7543
Epoch 4/40
30/30 [==============================] - 1s 18ms/step - loss: 0.6609 - accuracy: 0.7690 - val_loss: 0.6527 - val_accuracy: 0.7642
Epoch 5/40
30/30 [==============================] - 1s 17ms/step - loss: 0.6361 - accuracy: 0.7845 - val_loss: 0.6263 - val_accuracy: 0.7664
Epoch 6/40
30/30 [==============================] - 0s 15ms/step - loss: 0.6038 - accuracy: 0.7999 - val_loss: 0.5941 - val_accuracy: 0.7821
Epoch 7/40
30/30 [==============================] - 0s 15ms/step - loss: 0.5663 - accuracy: 0.8125 - val_loss: 0.5580 - val_accuracy: 0.8039
Epoch 8/40
30

## Avaliar o modelo

In [ ]:
results = model.evaluate(test_data, test_labels)

print(results)

782/782 [==============================] - 1s 1ms/step - loss: 1.3413 - accuracy: 0.5073
[1.3413382768630981, 0.5072799921035767]


O resultado que temos são dois numeros: erro (quanto menor, melhor) e a acurácia (quanto maior, melhor). Aqui podemos identificar que a accuracy é de 87%

## Grafico de erro e da acurácia

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

plt.clf()   # clear figure
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()